In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams

rcParams["figure.max_open_warning"] = False

In [ ]:
def plot_performance(err_dict, xs=None, xlabel="", ylabel="error"):
    """ Plot error evolution over xs and as cdf. """
    if xs is None:
        xs = range(len(err_dict.values()[0]))
    
    from matplotlib.lines import Line2D
    markers = [m for m in list(Line2D.markers.keys()) if m not in [".", "", ","]]
    
    i = 0
    fig, axs = plt.subplots(1, 2, squeeze=False)
    fig.set_size_inches(10, 5)
    max_abs = 0
    for method, err_list in err_dict.items():
        markersize = 8 - i
        
        axs[0, 0].plot(
            xs,
            err_list,
            label=method,
            marker=markers[i],
            ls=":",
            markersize=markersize,
        )
        
        xvals = sorted(np.abs(err_list))
        yvals = np.linspace(0, 1, len(xvals))
        axs[0, 1].plot(
            xvals, yvals, label=method, marker=markers[i], ls=":", markersize=markersize
        )
        i += 1
        
        max_abs = max(max_abs, max(xvals))

    # axs[0, 0].set_yscale('log')
    axs[0, 0].set_ylim(-max_abs, max_abs)
    axs[0, 0].set_ylabel(ylabel)
    axs[0, 0].set_xlabel(xlabel)
    axs[0, 0].legend(loc="upper right")

    axs[0, 1].set_ylabel("cdf")
    axs[0, 1].set_xlabel("absolute "+ylabel)
    axs[0, 1].grid(which="both")
    axs[0, 1].legend(loc="lower right")
    return fig, axs

In [ ]:
from simulation import get_setup

distance = 5

fig, ax = plt.subplots()
fig.set_size_inches(5, 5)
for yaw_deg, marker in zip([0,15,30], ["o", "*", "x"]):
    source, mic_positions = get_setup(distance_cm=distance, yaw_deg=yaw_deg)
    source_image = [source[0], -source[1]]
    for i, mic in enumerate(mic_positions):
        label = f"mic{i}" if yaw_deg == 0 else None
        ax.scatter(*mic*100, label=label, marker=marker, color=f"C{i}")
    ax.scatter(*mic*100, label=f"yaw {yaw_deg}deg", marker=marker, color=f"C{i}")
ax.plot([4.8*100, 5.2*100], [0, 0], color="k", label="wall")
ax.set_title(f"setup for distance={distance}cm and different angles")

handles, labels = ax.get_legend_handles_labels()
h_mics = {l: h for h, l in zip(handles, labels) if 'mic' in l}
l1 = ax.legend(h_mics.values(), h_mics.keys(),loc="lower right", bbox_to_anchor=[.3, 0])

h_other = {l: h for h, l in zip(handles, labels) if not 'mic' in l}
ax.legend(h_other.values(), h_other.keys(),loc="lower left", bbox_to_anchor=[.3, 0])
ax.add_artist(l1)

ax.set_xlabel("x [cm]")
ax.set_ylabel("y [cm]")
ax.axis("equal")

fig.savefig(f'plots/setup.pdf', bbox_inches='tight')

# Experimental distance-frequency matrix

In [ ]:
# exp_name = '2021_02_09_wall_tukey';
# exp_name = '2021_02_09_wall';
#exp_name = "2021_02_23_wall" # old buzzer
#exp_name = "2021_02_25_wall" # new buzzer
#exp_name = "2021_03_01_flying"
exp_name = '2021_02_19_windows';
# exp_name = '2020_12_9_rotating';
# exp_name = '2020_11_26_wall';
# fname = f'results/{exp_name}_real.pkl'
fname = f"../experiments/{exp_name}/all_data.pkl"
# fname = f'../experiments/{exp_name}/battery_data.pkl' # for 2021_02_09_wall, battery study

try:
    df_all = pd.read_pickle(fname)
    print("read", fname)
except Exception as e:
    print(e)
    print("Error: run wall_analysis.py to parse experiments.")
df_all.iloc[:, :8]

# plot_spectrograms(df_freq)

# 0. Debug parsing

In [ ]:
from bin_selection import generate_sweep
from crazyflie_description_py.parameters import N_BUFFER, FS
from dataset_parameters import kwargs_datasets
from pandas_utils import filter_by_dict
from plotting_tools import *
import seaborn as sns
from wall_detector import WallDetector, normalized_std
from frequency_analysis import get_index_matrix

mag_thresh = 1.0 # tested for 25_* and 23_* dataset
ymin = 1e-4
ymax = 3e2
degree = 0
verbose = False

if len(df_all.distance.unique()) > 1:
    distance = 20
else:
    distance = df_all.iloc[0].distance
    print('using unique distance', distance)
chosen_dict = {
    "degree": degree, 
    "distance": distance, 
    "mic_type": "audio_deck",
    "appendix": "_window2"
}
df_filtered = filter_by_dict(df_all, chosen_dict)
print("plotting\n", df_filtered.iloc[:, :8])

filter_by_motors = filter_by_mic_type = filter_by_appendix = False
if len(df_filtered.motors.unique()) > 1:
    filter_by_motors = True
if len(df_filtered.mic_type.unique()) > 1:
    filter_by_mic_type = True
if len(df_filtered.appendix.unique()) > 1:
    filter_by_appendix = True

for mic_type, df_mic_types in df_filtered.groupby("mic_type"):
    if mic_type == "measurement":
        mic = 0
    else:
        mic = 1
        
    for j, (i_row, row) in enumerate(df_mic_types.iterrows()):
        if (mic_type == "measurement") and (row.snr == 1):  # this is no "real snr"
            continue

        label = ""
        if filter_by_mic_type:
            label += f"{mic_type}, "
        if filter_by_motors:
            label += f"motors={row.motors}"
        if filter_by_appendix:
            label += f"{row.appendix.replace('_', ' ')} "
        label += f"mic{mic}"

        wall_detector = WallDetector.init_from_row(exp_name, row)
        spec_all = np.abs(wall_detector.current_spectrogram)
        freqs = wall_detector.current_freqs
        
        # plotting
        index_matrix = get_index_matrix(spec_all)
        spec = np.mean(spec_all, axis=1)
        
        fig, ax = plt.subplots()
        fig.set_size_inches(5, 5)
        log_spec = np.full(spec.shape, np.nan)
        log_spec[spec>0] = np.log10(spec[spec>0])
        im = ax.pcolorfast(row.seconds, freqs, log_spec[:-1, :-1])
        add_colorbar(fig, ax, im)
        ax.scatter(row.seconds[1:], 
                   freqs[index_matrix[0, :]][:-1], 
                   color='C1', ls=':', label='highest frequency', s=5)
        ax.legend()
        ax.set_xlim(
            kwargs_datasets[exp_name][row.mic_type]["min_time"],
            kwargs_datasets[exp_name][row.mic_type]["max_time"],
        )
        ax.set_ylim(
            kwargs_datasets[exp_name][row.mic_type]["min_freq"],
            kwargs_datasets[exp_name][row.mic_type]["max_freq"],
        )
        ax.set_xlabel("time [s]")
        ax.set_ylabel("frequency [Hz]")
        ax.set_title(label)
        #save_fig(
        #    fig, f"plots/{exp_name}_{row.mic_type}_{row.motors}{row.appendix}_spec.png"
        #)

        fig_raw, ax = plot_raw_signals(spec_all, freqs, mic_idx=mic)
        ax.set_ylim(ymin, ymax)
        ax.set_title(label)
        save_fig(
            fig_raw, f"plots/{exp_name}_{row.mic_type}_{row.motors}{row.appendix}_raw.png"
        )

## interpolation study

current status: interpolation is not helping much because we need to consider the wider shape of the peak (due to windowing) in the pipeline somewhere. The fit_peak function might be a better solution but it is also not quite working yet. 

In [ ]:
import time
from frequency_analysis import interpolate_peak, fit_peak

mic_idx = 0
chosen_dict = {
    "degree": degree, 
    "distance": distance, 
    "mic_type": "audio_deck",
    "appendix":"_window2",
    "motors":"all45000"
}
df_filtered = filter_by_dict(df_all, chosen_dict)
assert len(df_filtered) == 1
row = df_filtered.iloc[0]
wall_detector = WallDetector.init_from_row(exp_name, row)

spec = wall_detector.current_spectrogram
freqs = wall_detector.current_freqs

n_extra = 12
time_range = list(range(282, 291))
fig, ax = plt.subplots()
plot_range = list(
    range(time_range[0]-n_extra, time_range[0]) 
) + time_range + list(
    range(time_range[-1]+1, time_range[-1]+n_extra+1) 
)
spec_plot =  np.abs(spec[:, mic_idx, plot_range])
freqs_valid = np.any(spec_plot, axis=1)
spec_plot = spec_plot[freqs_valid]
im = ax.pcolorfast(plot_range, freqs[freqs_valid], spec_plot[:-1, :-1])
add_colorbar(fig, ax , im)

In [ ]:
fig, axs = plt.subplots(2, 1, sharex=True)
fig.set_size_inches(10, 5)

for t_idx in time_range:
    spec_slice = spec[:, mic_idx, t_idx]

    max_without = np.max(np.abs(spec_slice))
    #print(f'magniutde estimate without fitting: {max_without:.2f}')

    # use full slice
    max_val, max_freq = interpolate_peak(spec_slice, freqs)#, ax=ax)
    
    #print(f'magniutde estimate with interpolation with all points: {max_val:.2f}')
    phase = np.angle(spec_slice)
    axs[0].plot(freqs, np.abs(spec_slice), label=t_idx)
    axs[1].plot(freqs, phase, label=t_idx)
    #ax.axvline(max_freq, color='C1')
    axs[0].set_xlim(max_freq-500, max_freq+500)
    axs[0].set_xlim(5400, 5700)

    # use slice around max
    bin_max = np.argmax(np.abs(spec_slice))
    n_points = 2
    range_max = range(bin_max-n_points, bin_max+n_points)
    spec_smaller = spec_slice[range_max]
    freq_smaller = freqs[range_max]

    max_val_smaller, max_freq_smaller = interpolate_peak(spec_smaller, freq_smaller, pad_factor=2)
    #print(f'magniutde estimate with interpolation with {n_points} points: {max_val_smaller:.2f}')
    #ax.plot(freqs, np.abs(spec_slice))
    #ax.axvline(max_freq_smaller, color='C2')
    #ax.set_xlim(max_freq_smaller-500, max_freq_smaller+500)

    max_peak, p = fit_peak(np.abs(spec_slice))
    #print(f'magniutde estimate with peak fitting: {max_peak:.2f}')
axs[0].legend()

In [ ]:
chosen_dict = {
    "degree": degree,
    "distance": distance,
    "mic_type": "audio_deck",
}
df_filtered = filter_by_dict(df_all, chosen_dict)

if mic_type == "measurement":
    mic = 0
else:
    mic = 1

n_rows = len(df_filtered.appendix.unique())
print(n_rows)
fig_slice, ax_slices = plt.subplots(n_rows, 2, sharex=True, sharey=True, squeeze=False)
fig_slice.set_size_inches(10, 5 * n_rows)

for i, (appendix, df) in enumerate(df_filtered.groupby("appendix", sort=True)):
    for j, (motors, df_row) in enumerate(df.groupby("motors", sort=True)):
        assert len(df_row) == 1, df_row.iloc[:, :10]
        row = df_row.iloc[0]
        if i == 0:
            ax_slices[0, j].set_title(motors)

        t0 = time.time()
        wall_detector = WallDetector.init_from_row(exp_name, row, interpolation='')
        t1 = time.time()
        wall_detector_lagrange = WallDetector.init_from_row(exp_name, row, interpolation='lagrange')
        t2 = time.time()
        wall_detector_quadratic = WallDetector.init_from_row(exp_name, row, interpolation='quadratic')
        t3 = time.time()
        print(f'times with interpolation:\n none:{t1-t0:.2e}s \n zero-pad:{t2-t1:.2e}s \n quadratic:{t3-t2:.2e}')

        for idx, wd in enumerate([wall_detector, wall_detector_lagrange, wall_detector_quadratic]):
            df_mic = wd.df[wd.df.mic == mic]
            label = f"interpolation {wd.interpolation}"
            stds = (
                df_mic.groupby("frequency", sort=True).magnitude.apply(np.nanstd).values
            )
            medians = (
                df_mic.groupby("frequency", sort=True)
                .magnitude.apply(np.nanmedian)
                .values
            )
            xs = np.sort(df_mic.frequency.unique())
            ax_slices[i, j].plot(xs, stds, label=label)
            
            #stds = np.r_[[np.zeros(len(medians))], [10 * stds]]
            #dx_mean = np.mean(xs[:-1]-xs[1:])
            #ax_slices[i, j].errorbar(
            #    x=xs+idx*dx_mean/5,
            #    y=medians,
            #    yerr=stds,
            #    label=label,
            #    marker="o",
            #    markersize=4.0,
            #    lw=2.0,
            #    alpha=1-idx*0.3
            #)
        ax_slices[i, j].grid(which="both")
        # sns.scatterplot(
        #    data=df_mic, x="frequency", y="magnitude", hue="counter", ax=ax, linewidth=0
        # )
        ax_slices[i, j].set_yscale('log')
    ax_slices[i, -1].legend(loc='upper right')
    title = appendix.replace("_", "")
    ax_slices[i, 0].set_ylabel(title)
    break
ax_slices[-1, 0].set_xlabel("frequency [Hz]")
ax_slices[-1, 1].set_xlabel("frequency [Hz]")
#save_fig(fig_slice, f"plots/{exp_name}_slice.png")

# Full analysis

## 1. Regenerate or read results

In [ ]:
from wall_analysis import get_df_matrices, FILTERS

exp_name = "2021_02_25_wall"
#exp_name = "2021_02_23_wall"

fname = "results/wall_analysis.pkl"
# method = 'dryrun'
# method = 'new'
#method = 'replace'
method = "read"

if method == "read":
    results_df = pd.read_pickle(fname)
elif method == "replace":
    from pandas_utils import fill_df

    FILTERS = ["mic_type", "snr", "motors"]
    results_df = pd.read_pickle(fname)

    print("read", fname)
    results_df_new = get_df_matrices(exp_name)

    for chosen_tuple, df in results_df_new.groupby(FILTERS):
        assert len(df) == 1
        row = df.iloc[0]

        filter_dict = dict(zip(FILTERS, chosen_tuple))
        filter_dict.update({"exp_name": exp_name})
        fill_dict = {
            "df_dist": row.df_dist,
            "df_freq": row.df_freq,
            "df_matrix": row.df_matrix,
        }
        print("filling results_df_old...")
        results_df = fill_df(results_df, filter_dict, fill_dict)
    pd.to_pickle(results_df, fname)
    print("saved as", fname)
elif method == "new":
    results_df = get_df_matrices(exp_name)
    pd.to_pickle(results_df, fname)
    print("saved as", fname)
elif method == "dryrun":
    results_df = get_df_matrices(exp_name, max_distance=10, plot=True)
results_df

## 2. Pick dataset

In [ ]:
from pandas_utils import filter_by_dict
#exp_name = "2021_02_23_wall" # old buzzer
exp_name = "2021_02_25_wall" # new buzzer

# mic_type = 'measurement'
mic_type = "audio_deck"
#motors = 'all45000'
motors = 0 

fname = f"{exp_name}_{mic_type}_{motors}"

filter_dict = dict(exp_name=exp_name, motors=motors, mic_type=mic_type)

df = filter_by_dict(results_df, filter_dict)
assert len(df) == 1
row = df.iloc[0]

row

In [ ]:
from wall_detector import prune_df_matrix


df_matrix, df_freq, indices = prune_df_matrix(
    row.df_matrix, row.df_freq, ratio_missing_allowed=0.2, verbose=True
)

fig, ax_all = plt.subplots()
for mic_idx in range(4):
    fig, ax = plt.subplots()
    ax.pcolormesh(row.df_dist, row.df_freq, row.df_matrix[mic_idx])
    fig, ax = plt.subplots()
    ax.pcolormesh(row.df_dist, df_freq, df_matrix[mic_idx])
    ax_all.plot(df_freq, np.mean(df_matrix[mic_idx], axis=1), label=f'mic{mic_idx}')

In [ ]:
from calibration import get_calibration_function, get_calibration_function_dict
from wall_detector import normalize_df_matrix
from copy import deepcopy
from simulation import get_df_theory

fig, ax = plt.subplots()
calib_function_old = get_calibration_function(ax=ax)
ax.set_title("old calibration")
fig, ax = plt.subplots()
calib_function = get_calibration_function_dict(**filter_dict, ax=ax)
ax.set_ylim(1, 10)
ax.set_yscale("linear")
ax.set_title("new calibration")

# 1. study distance slices

In [ ]:
def fill_nans(slice_f, xs):
    for m in range(slice_f.shape[0]):
        mask = np.isfinite(slice_f[m])
        slice_f[m, :] = np.interp(xs, xs[mask], slice_f[m, mask])
    return slice_f

In [ ]:
from simulation import get_dist_slice_theory

def plot_ffts(slice_exp, slice_the, dist):
    fig_f, axs_f = plt.subplots(1, slice_exp.shape[0], squeeze=False, sharey=True)
    fig_f.set_size_inches(10, 5)
    fig_f.suptitle(f"FFT of standardized distance slices, at frequency {f:.0f}Hz")
    for m in range(slice_exp.shape[0]):
        slice_exp_norm = deepcopy(slice_exp[m])
        slice_the_norm = deepcopy(slice_the[m])
        slice_exp_norm -= np.mean(slice_exp_norm)
        slice_the_norm -= np.mean(slice_the_norm)

        n = max(len(slice_exp_norm), 1000)
        freqs = np.fft.rfftfreq(n, d=dist[1] - dist[0])  # unit: 1/cm
        fft_exp = np.abs(np.fft.rfft(slice_exp_norm, n=n))
        fft_exp /= np.sum(fft_exp)
        fft_theory = np.abs(np.fft.rfft(slice_the_norm, n=n))
        fft_theory /= np.sum(fft_theory)

        axs_f[0, m].plot(freqs, fft_theory, label="theoretical")
        axs_f[0, m].plot(freqs, fft_exp, label="measured")
        axs_f[0, m].set_title(f"mic{m}")
        axs_f[0, m].legend(loc="upper right")
    return fig_f

valid_freqs = df_freq[~np.any(np.isnan(df_matrix), axis=(0, 2))]
plot_freqs = [valid_freqs[0], valid_freqs[len(valid_freqs) // 2], valid_freqs[-2]]
for i, f in enumerate(df_freq):
    slice_exp = df_matrix[:, i, :]
    if np.any(np.isnan(slice_exp)):
        slice_exp = fill_nans(slice_exp, row.df_dist)
    slice_the = get_dist_slice_theory(f, row.df_dist).T
    
    fig_f = plot_ffts(
        slice_exp=slice_exp,
        slice_the=slice_the,
        dist=row.df_dist,
    )
    break

In [ ]:
from calibration import fit_distance_slice

fitting_results = pd.DataFrame(columns=['frequency', 'mic', 'absorption', 'gain', 'offset', 'median', 'method'])

valid_freqs = df_freq[~np.any(np.isnan(df_matrix), axis=(0, 2))]
plot_freqs = [valid_freqs[0], valid_freqs[len(valid_freqs) // 2], valid_freqs[-2]]

chosen_mics = range(4)

for method in ['brute', 'minimize']:
    times = []
    for i, f in enumerate(df_freq):
        slice_exp = df_matrix[:, i, :]
        if np.any(np.isnan(slice_exp)):
            slice_exp = fill_nans(slice_exp, row.df_dist)

        #print("attenuation, absorption, gain, distance offset")
        fig, axs = plt.subplots(1, len(chosen_mics), sharey=True, squeeze=False)
        fig.set_size_inches(5*len(chosen_mics), 5)
        for i, mic_idx in enumerate(chosen_mics):
            # heuristic
            gain_x = np.median(slice_exp[mic_idx, :])
            coeffs_h = [0.2, gain_x, 0]
            slice_heur = get_dist_slice_theory(
                f,
                row.df_dist+coeffs_h[2],
                yaw_deg=0,
                chosen_mics=[mic_idx],
                wall_absorption=coeffs_h[0],
                gain_x=coeffs_h[1]
            )
            t0 = time.time()
            coeffs, slice_calib, cost = fit_distance_slice(
                slice_exp.T, row.df_dist, yaw_deg=0, frequency=f, chosen_mics=[mic_idx],
                method=method
            )
            times.append(time.time() - t0)
            axs[0, i].plot(row.df_dist, slice_exp[mic_idx, :], label="experimental")
            axs[0, i].plot(row.df_dist, slice_calib[:, 0], label="calibrated")
            #axs[0, i].plot(row.df_dist, slice_heur[:, 0], label="heuristic")
            axs[0, i].set_title(f'mic{mic_idx}')
            fitting_results.loc[len(fitting_results), :] = dict(
                frequency=f,
                mic=mic_idx,
                absorption=coeffs[0],
                gain=coeffs[1],
                offset=coeffs[2],
                median=gain_x,
                method=method
            )
        fig.suptitle(f'{f}Hz')
        axs[0, i].legend()

        #if f in plot_freqs:
            #save_fig(fig, f"plots/{fname}_distance_slice_{f:.0f}.pdf")
    print(f'average time {method}: {np.mean(times):.2f}s')

In [ ]:
color_palette = 'tab10'
extra_kwargs = {
    'linewidth':0,
    'palette':color_palette,
    'style': 'method',
    'hue': 'mic',
    'data': fitting_results,
    'x': 'frequency',
}
palette = sns.palettes.color_palette(color_palette)
fitting_results = fitting_results.apply(pd.to_numeric, errors='ignore', axis=0)
plt.figure()
sns.scatterplot(y='gain', **extra_kwargs, )
plt.figure()
sns.scatterplot(y='median', **extra_kwargs)

#plt.figure()
#sns.scatterplot(y='absorption', **extra_kwargs)

plt.figure()
sns.scatterplot(y='offset', **extra_kwargs)

ls = {'brute':'-', 'minimize':':'}
for method, df in fitting_results.groupby('method'):
    medians = df.groupby(['mic']).offset.median()
    [plt.axhline(m, color=palette[i], ls=ls[method]) for i, m in enumerate(medians)]
    plt.plot([], [], color='black', ls=ls[method], label=method)
plt.legend(loc='upper left', bbox_to_anchor=[1, 1])
pass

In [ ]:
from simulation import factor_distance_to_delta

for mic in range(4):
    print('mic', mic)
    for distance in np.arange(10, 50, step=10):
        f = factor_distance_to_delta(distance, mic)
        print(distance, f)

In [ ]:
# distance slice algorithm
from wall_detector import get_approach_angle_fft

gt_gamma = 90 # in degrees
err_dict = {f"mic{m}": [np.nan]*len(df_freq) for m in range(df_matrix.shape[0])}
for i, f in enumerate(df_freq):
    
    plt.figure()
    for mic_idx in range(df_matrix.shape[0]):
        d_slice = df_matrix[mic_idx, i, :] 
        gammas, prob = get_approach_angle_fft(d_slice, f, row.df_dist, mic_idx,
                                              n_max=1000, bayes=False)
        gamma = gammas[np.argmax(prob)]
        plt.plot(gammas, prob)
        plt.axvline(gamma, label=f'mic{mic_idx}: $\\gamma$={gamma:.1f}', color=f'C{mic_idx}')
        err_dict[f"mic{mic_idx}"][i] = gamma - gt_gamma
        
    plt.title(f'frequency {f:.0f}Hz')
    plt.xlabel('angle of approach $\\gamma$ [deg]')
    plt.ylabel('probability')
    plt.legend()
        
fig, axs = plot_performance(err_dict, xs=df_freq, 
           xlabel="frequency [Hz]", ylabel="error [deg]")
save_fig(fig, f'plots/{fname}_distance_slice_performance.pdf')

# 2. study frequency slices

In [ ]:
min_value = np.inf
max_value = -np.inf

distances_grid = np.arange(0, max(row.df_dist) + 1)
distances_idx = np.argmin(np.abs(distances_grid[:, None] - row.df_dist[None, :]), axis=0)

results = pd.DataFrame(columns=["normalization", "matrix", "values"])
method_dict = {
    'raw': "",
    'normalize': "normalize",
    'calibration-offline-old': calib_function_old,
    'calibration-offline': calib_function,
    'calibration-online': "calibration-online",
    'standardize': "standardize",
    'zero_mean': "zero_mean",
    'theoretical': "theoretical",
}

if row.mic_type == "measurement":
    plot_channel_idx = 0  # only choice
else:
    plot_channel_idx = 1  # corresponds to mic below measurement mic

__, df_freq, indices = prune_df_matrix(row.df_matrix, row.df_freq)
df_theory_pruned = df_theory[:, indices, :]

for j, (key, method) in enumerate(method_dict.items()):
    values = None
    df_matrix = row.df_matrix[:, indices, :]

    if key == "raw":
        df_norm = deepcopy(df_matrix)
    elif key == "theoretical":
        df_norm = deepcopy(df_theory_pruned[:, :, distances_idx])
    else:
        df_norm, values = normalize_df_matrix(
            df_matrix=df_matrix, freqs=df_freq, method=method
        )
    results.loc[len(results), :] = {
        "normalization": key,
        "matrix": df_norm,
        "values": values,
    }
    # print(key, data_type, df_norm.shape)
    min_value = min(min_value, -abs(np.min(df_norm)))
    max_value = max(max_value, abs(np.max(df_norm)))

In [ ]:
from calibration import get_calibration_function_matrix

fig, ax = plt.subplots()
get_calibration_function_matrix(row.df_matrix, row.df_freq, ax=ax)

In [ ]:
min_freq = min(df_freq)
max_freq = max(df_freq)

df_the = df_theory_pruned[plot_channel_idx, :, distances_idx].T
min_value = None
max_value = None

for normalization, df in results.groupby("normalization", sort=False):
    df_exp = df.iloc[0].matrix[plot_channel_idx]

    fig, ax = plt.subplots()
    fig.set_size_inches(5, 5)
    ax, im = plot_df_matrix(
        row.df_dist,
        df_freq,
        df_exp,
        ax=ax,
        min_freq=min_freq,
        max_freq=max_freq,
        vmin=min_value,
        vmax=max_value,
    )
    add_colorbar(fig, ax, im)
    ax.set_title(f"{normalization}")
    ax.set_xlabel('distance [cm]')
    ax.set_ylabel('frequency [Hz]')
    
    save_fig(fig, f"plots/{fname}_matrices_{normalization}.png")

# 3. Visualize cost function

In [ ]:
from simulation import get_freq_slice_theory
from wall_detector import get_probability_cost, get_probability_fft

mic_idx = 1

# chosen_methods = results.normalization.unique()
chosen_methods = ["raw", "calibration-online", "standardize", "theoretical"]

# TODO(FD) actually choose good frequencies here
chosen_frequencies = df_freq

freq_indices = np.where(chosen_frequencies[None, :] == df_freq[:, None])[0]
np.testing.assert_allclose(df_freq[freq_indices], chosen_frequencies)

absolute_err_dict = {
    key: {method: [] for method in chosen_methods} for key in ["cost", "fft"]
}

distances = row.df_dist[row.df_dist <= max_distance]

plot_distances = [2, 5, 10]

for i_d, distance in enumerate(distances):

    fig, ax = plt.subplots()
    fig.set_size_inches(5, 5)
    fig_cost, ax_cost = plt.subplots()
    fig_cost.set_size_inches(5, 5)
    fig_fft, ax_fft = plt.subplots()
    fig_fft.set_size_inches(5, 5)
    
    for i_method, method in enumerate(chosen_methods):
        df = results.loc[results.normalization == method]
        slice_exp = df.iloc[0].matrix[[mic_idx], freq_indices, i_d]

        # doing this here for plotting reasons. Doesn't change performance as
        # this is done again in get_probability_cost
        slice_exp -= np.mean(slice_exp)
        slice_exp /= np.std(slice_exp)

        proba_cost = get_probability_cost(
            slice_exp, df_freq[freq_indices], 
            distances_grid, mic_idx=mic_idx
        )
        distances_fft, proba_fft = get_probability_fft(
            slice_exp, 
            df_freq[freq_indices]
        )

        d_cost_idx = np.argmax(proba_cost)
        d_cost = distances_grid[d_cost_idx]
        abs_err = abs(d_cost - distance)
        absolute_err_dict["cost"][method].append(abs_err)

        ax_cost.semilogy(distances_grid, proba_cost, color=f"C{i_method}", label=method)
        ax_cost.axvline(x=d_cost, color=f"C{i_method}")
        ax_cost.axvline(x=distance, color="black", ls=":")
        ax_cost.set_xlabel("distances [cm]")
        ax_cost.set_ylabel("probability")
        ax_cost.set_xlim(min(distances_grid), max(distances_grid))
        ax_cost.set_ylim(1e-5, 1)
        ax_cost.set_title(f"probability for distance {distance:.0f}cm")
        
        d_fft_idx = np.argmax(proba_fft)
        d_fft = distances_fft[d_fft_idx]
        absolute_err_dict["fft"][method].append(abs(d_fft - distance))
        

        ax_fft.semilogy(distances_fft, proba_fft, color=f"C{i_method}", ls=":")
        ax_fft.axvline(x=d_fft, color=f"C{i_method}", ls=":")
        ax_fft.axvline(x=distance, color="black", ls=":")
        ax_fft.set_xlim(min(distances_grid), max(distances_grid))
        ax_fft.set_ylim(1e-5, 1)
        ax_fft.set_xlabel("distances [cm]")
        ax_fft.set_ylabel("probability")
        ax_fft.set_title(f"probability for distance {distance:.0f}cm")

        # plotting
        ax.plot(chosen_frequencies, slice_exp, label=method, color=f"C{i_method}")
        ax.set_xlim(min_freq, max_freq)
        ax.set_ylim(-3, 3)
        ax.set_xlabel("frequency [Hz]")
        ax.set_ylabel("amplitude")
        ax.set_title(f"slices for distance {distance:.0f}cm")
        

    ax.legend(loc="lower right")
    ax_cost.legend(loc="lower right")

    if distance in plot_distances:
        fname_here = f'plots/{fname}_{distance:.0f}_slice.png'
        save_fig(fig, fname_here)
        
        fname_here = f'plots/{fname}_{distance:.0f}_cost.png'
        save_fig(fig_cost, fname_here)

# 4. Evaluate algorithm performance

In [ ]:
for key, absolute_err in absolute_err_dict.items():
    fig, axs = plot_performance(absolute_err, xs=distances, xlabel="distance [cm]", ylabel="absolute error [cm]")
    axs[0, 1].set_xlim(-1, max(distances))
    # fname = f'results/{exp_name}/performance_{mic_type}_snr{snr}_subset.png'
    # fname = f'results/{exp_name}/performance_{mic_type}_snr{snr}_all.png'
    fname_here = f"plots/{fname}_{key}_frequency_performance.png"
    save_fig(fig, fname_here)

In [ ]:
# old stuff

In [ ]:
from scipy.interpolate import UnivariateSpline, LSQUnivariateSpline
from numpy.polynomial import Chebyshev, Legendre, Polynomial, Laguerre

spline_k = 2
spline_ext = 3

for mic_idx in range(row.df_matrix.shape[0]):
    df_norm, values = normalize_df_matrix(
        df_matrix=row.df_matrix, freqs=row.df_freq, method="calibration-online"
    )

    df_norm, df_freq, indices = prune_df_matrix(df_norm, row.df_freq)
    values = values[:, indices]

    xvalues = df_freq
    yvalues = np.log10(values[mic_idx, :, 0])

    plt.figure()
    plt.plot(xvalues, yvalues, label="raw", color="C0", marker="*")
    for i, poly in enumerate([Legendre]):  
        c = poly.fit(xvalues, yvalues, deg=10)
        x, y = c.linspace()
        plt.plot(x, y, label=poly.__name__, color=f"C{i+1}")

    # spline = UnivariateSpline(xvalues, yvalues, k=2)
    # plt.plot(xvalues, spline(xvalues), label='Spline', color='C5')

    knots = np.linspace(min(df_freq), max(df_freq), 7)[1:-1]
    spline = LSQUnivariateSpline(xvalues, yvalues, t=knots, k=spline_k, ext=spline_ext)
    plt.plot(xvalues, spline(xvalues), label="LSQSpline", color=f"C{i+2}", marker="o")
    [plt.axvline(k, color=f"C{i+2}") for k in knots]
    plt.legend()
    plt.title(f"mic{mic_idx}")

In [ ]:
def spline_interpolation(xvalues, df_matrix):
    """ Generate spline interpolation for all distances.
    
    :return calib_values: mics x freqs x distances, interpolated spline curve
    """
    knots = np.linspace(min(df_freq), max(df_freq), 6)[1:-1]
    calib_values = np.empty_like(df_matrix)
    for i, dist in enumerate(row.df_dist):
        for m in range(df_matrix.shape[0]):

            slice_f = np.log10(df_matrix[m, :, i])
            mask = ~np.isnan(slice_f)

            spline = LSQUnivariateSpline(
                xvalues[mask], slice_f[mask], t=knots, k=spline_k, ext=spline_ext
            )
            calib_values[m, :, i] = 10 ** spline(xvalues)
    return calib_values


df_matrix, df_freq, __ = prune_df_matrix(row.df_matrix, row.df_freq)
calib_values = spline_interpolation(df_freq, df_matrix)

fig, axs = plt.subplots(1, calib_values.shape[0], squeeze=False, sharey=True)
fig.set_size_inches(10, 5)
cmap = plt.get_cmap("inferno")
for m in range(calib_values.shape[0]):
    for i, dist in enumerate(row.df_dist):
        axs[0, m].semilogy(
            df_freq, calib_values[m, :, i], color=cmap(i / len(row.df_dist))
        )
    axs[0, m].semilogy(
        df_freq, calib_values[m, :, 0], color=cmap(0), label=f"{row.df_dist[0]}cm"
    )
    axs[0, m].semilogy(
        df_freq,
        calib_values[m, :, i],
        color=cmap(i / len(row.df_dist)),
        label=f"{row.df_dist[-1]}cm",
    )
    axs[0, m].set_title(f"mic{m}")
    axs[0, m].legend(loc="lower right")